## nomad-parser-nexus demo

Sprint5, EM

### Step -1: Set up dependencies for jupyterlab_h5web

<p style="color:red">Instructions how to set up all dependencies to start a new virtualenv with a jupyterlab installation.</p>

In [ ]:
# start with a fresh virtualenv
# ! pip install virtualenv
# ! virtualenv --python=python3.7 .py37env
# ! source .py37env/bin/activate

# install jupyter, jupyter-lab and web extensions

#inside SPRINT5-JUPYTER-TEST-02, conda, python 3.7, jupyter, jupyterlab

In [ ]:
# ! pip install --upgrade nodejs && pip install ipywidgets h5py==3.5.0 h5glance==0.7 h5grove==0.0.8 jupyterlab[full]==2.3.0 jupyterlab_h5web[full]==0.0.11 punx==0.2.5 nexpy==0.14.1 silx[full] && jupyter lab build

In [ ]:
# ! jupyter nbextension enable --py widgetsnbextension

In [ ]:
# ! jupyter serverextension enable jupyterlab_h5web

<p style="color:red">Restart the Jupyter kernel, or easier start the following code after you have setup these environment steps.</p>

### Step 0: Installating and testing nomad-parser-nexus module

In [ ]:
! pip list && pip install --upgrade pip && pip install nomad-lab==1.0.0 --extra-index-url https://gitlab.mpcdf.mpg.de/api/v4/projects/2187/packages/pypi/simple

In [ ]:
! git clone https://github.com/nomad-coe/nomad-parser-nexus.git --branch yaml2nxdl --recursive parser-nexus && cd parser-nexus && git status && pip install -r requirements.txt
# once yaml2nxdl was merged with master
# ! git clone https://github.com/nomad-coe/nomad-parser-nexus.git --recursive parser-nexus && cd parser-nexus && git status && pip install -r requirements.txt

In [ ]:
# ! cd parser-nexus && git status && git pull && pip install -r requirements.txt

In [ ]:
! cd parser-nexus && pip install -e .[all]
! pip list | grep nomad*
! pip list | grep nexus*

In [ ]:
# in the above cells clear redundant commands based on todays history
! cd parser-nexus && pytest -sv tests

### Step 1: Download example data (for EM)

In [2]:
# showing here the example for APM, http://dx.doi.org/10.5281/zenodo.5911409
import shutil
! curl --output EM.STEM.Nion.Datasets.3.zip https://zenodo.org/record/5911409/files/EM.STEM.Nion.Datasets.3.zip
shutil.unpack_archive('EM.STEM.Nion.Datasets.3.zip')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  498k  100  498k    0     0   361k      0  0:00:01  0:00:01 --:--:--  361k


### Step 2: Run your \<\<YOUR_METHOD\>\>-specific dataconverter/readers/\<\<YOUR_READER\>\> on your \<\<EXAMPLE_DATA\>\>

In [10]:
#parser-nexus/tests/data/tools/dataconverter/readers/em/
! python parser-nexus/nexusparser/tools/dataconverter/convert.py --reader em_nion --nxdl parser-nexus/nexusparser/definitions/applications/NXem_nion.nxdl.xml \
--input-file HAADF_01.npy \
--input-file HAADF_01.ELabFTW.dat \
--input-file HAADF_01.json --output em3.test.nxs

Using em_nion reader to convert the given files:  
• HAADF_01.npy
• HAADF_01.ELabFTW.dat
• HAADF_01.json 
Add metadata which come from other sources...
Extracting data from NionSwift JSON file: HAADF_01.json
Loading: NionSwiftJsonToNexusTranslationTable.ods from...
/home/mkuehbach/SPRINT5-JUPYTER-TEST-03/readers/em_nion/utils/
Add metadata/data from numpy array(s) representing scans...
Extracting data from NionSwift NPY file: HAADF_01.npy
Add metadata from e.g.ELN/LIMS dump JSON files...
Extracting data from ELN/LIMS/others JSON file: HAADF_01.ELabFTW.dat
The output file generated: em3.test.nxs


**The key take home message is that the above-specified command triggers the automatic creation of the HDF5 file.** This *.nxs file, is an HDF5 file.

### Step 3: Inspect the HDF5/NeXus file em3.test.nxs using H5Web

In [5]:
! jupyter serverextension list
! jupyter labextension list

config dir: /home/mkuehbach/.jupyter
    jupyterlab_h5web  enabled 
    - Validating...
      jupyterlab_h5web  OK
config dir: /home/mkuehbach/SPRINT5-JUPYTER-TEST-03/.pyenv/etc/jupyter
    jupyterlab_h5web  enabled 
    - Validating...
      jupyterlab_h5web  OK
    jupyterlab  enabled 
    - Validating...
      jupyterlab 2.3.0 OK
JupyterLab v2.3.0
Known labextensions:
   app dir: /home/mkuehbach/SPRINT5-JUPYTER-TEST-03/.pyenv/share/jupyter/lab
        jupyterlab-h5web v0.0.11  enabled  OK


In [6]:
from jupyterlab_h5web import H5Web

In [7]:
# h5_file_name = 'parser-nexus/tests/data/nexus_test_data/201805_WSe2_arpes.nxs'
h5_file_name = 'em3.test.nxs'

In [8]:
H5Web(h5_file_name)

<jupyterlab_h5web.widget.H5Web object>

Here is where the general template ends. Continue to fill the notebook based on<br>
**your own** post-processing of the *.nxs file, taking e.g. inspiration from<br>
sprints 2 and 3 in the nomad-remote-tools-hub mpcdf git repo.

Congratulations. **Given that all NORTH tools have mounted your current directory, you can now inspect this HDF5 file.**<br>
**The clue is you can now use the data processing tools from NORTH for analysing apm.test.nxs further.**<br>
Again, these analysis tools come shipped with nomad so there is no need to install them locally.<br>